# Optimization Part 2: Multivariate Scalar Functions

We now move into minimizing objectives that are multivariate functions.  They still return a single quantity that we wish to minimize, so they are scalar functions, but the objective function in this case depends on *more than one* function input. That means that we have to find the minimum with respect to multiple function inputs. We encounter this type of problem all the time!

Let's start with a very basic example: we have a function that describes the value of z for values of x and y:

$$z(x,y) = x^2 + y^2$$

By inspection, we know that this function has a minimum at x = 0, y = 0, but if we can, it's a good idea to start with a visualization of the problem.  This is a 3D problem, so we are still able to visualize it reasonably well. Once we hit 4D, all bets are off!  

The following code will create a surface plot for visualization:
    
    using Plots
    x = range(-5, stop = 5, length = 100)
    y = range(-5, stop = 5, length = 100)
    z(x,y) = (x^2 + y^2)   
    plot(x, y, z, st = surface)

This is a multivariate function, z(x,y), that returns a single output, z. We can minimize z using optimize from the Optim.jl package, albeit with a different set of algorithms than Brent's and GoldenSection, which we use for univariate problems. 

The **optimize** routine in Julia will only work on a function that accepts a single argument as an input - it will vary that argument iteratively until it finds a minimum value of the objective function (z here). So, if I were to give z(x,y) to optimize, it has two arguments, and it would generate an error. Instead, I have to create a function that takes all of the parameters I want to vary as a single *vector* input  So, instead of working with function forms that have two inputs/arguments like:

    function z(x,y)
        zval = x^2 + y^2
    end
        
or it's analogous anonymous function

    z = (x,y) -> (x^2 + y^2)
    
We want to write this particular objective function such that it takes a single vector argument that is a collection of all of the variables we want to minimize with respect to:

    function z(var)
        retval = var[1]^2 + var[2]^2
    end

or, it's analgous anonymous function:

    z = var -> (var[1]^2 + var[2]^2)
    
In either case, "var" would be an array that contains the values var = [x,y]
    
Often, to improve readability in a complex function, I will use a long form function definition and redefine elements in "var" using the labels that are specific to the way we express our function on paper:

    function z(var)
        x = var[1]
        y = var[2]
        retval = x^2 + y^2
    end

But any of the above options will work. Now, we can pass this objective function to a minimizer.  The only other catch is that we need an initial guess for the (x,y) pair, and it should be a collection of the same size and type that we used for our function argument. So, my full optimization script would look something like this. Just so that we can make the optimizer work for it, let's start at an initial guess of [10., 50.] for x and y:
    
    function z(var)
        x = var[1]
        y = var[2]
        retval = x^2 + y^2
    end
    
    var0 = [10., 50.]  #my initial guess for x and y formatted as a 1D Array.
    solution = optimize(z, var0)
    println(solution)
    println(solution.minimizer)
    println(solution.minimum)
    
Note that I need an array of floats here; integer arrays will cause an error. This is why I added a decimal point after each guess.

Now that we are getting into more advanced problems, I want to introduce a method for "passing" extra parameters into an optimization routine. You often will encounter an optimization problem where you need to minimize an objective function by varying certain variables, but you also need to pass additional information to the function, and you do not want that information to be adjusted by the optimization routine.  As an example, let's use the following function:

$$q(x,y,a,b) = ax^2 + by^2 + x - y$$

In this particular, example, a and b are coefficients, while x and y are the true "independent variables" that we are looking to optimize this function with respect to; however, our solution also will generally depend on the values of a and b, and we may need to consider the solution for multiple values of a and b. It complicates matters that optim.optimize routines only work on a function that takes a single, vector input as its argument, yet we need to provide a and b to this objective as well.

This situation comes up **very often**. We handle it in two steps. First, we construct the following function, which takes var, a, and b as arguments. Note that var, again, contains x and y, which are the parameters that we want to vary with fminunc; a and b are sent as fixed parameters.

    function q(var, a, b)
        x = var[1]
        y = var[2]
        retval = a*x^2 + b*y^2 + x - y
    end
        
This is, in reality, what we want to minimize with respect to x and y; however, optimize routines will only accept a single function argument. To pass extra arguments (a and b in this case), we will convert this function into a new objective function that only takes "var" as an argument.  We do this using anonymous functions as shown below for the case where a = 1, b = 3:

    a = 1;
    b = 3;
    objective = var -> q(var, a, b) #This is converting q(var, a, b) into objective(var)
    solution = optimize(objective, var0)
    solution.minimizer
    
It might not yet be apparent why you'd want to do this, but keep working on optimization problems for awhile, and I'm sure you'll find some examples. I rarely optimize a function that *doesn't* use this convention these days.

You can build almost infinitely from this basic framework of optimizing multivariate functions.  You can easily extend to optimize with respect to 3, 4, 5, or 15 parameters; however, just be aware that the more parameters you have, the more difficult it is to perform a meaningful optimization. 